# Step 4: PyTorch 기초 - 프레임워크로 딥러닝 시작하기

지금까지 NumPy로 신경망을 직접 구현했습니다. 이제 PyTorch를 사용하여 더 효율적으로 딥러닝을 해봅시다!

## 학습 목표
1. PyTorch의 기본 개념 이해 (텐서, 자동 미분)
2. nn.Module로 신경망 구축하기
3. 옵티마이저와 손실 함수 사용하기
4. GPU 활용하기
5. 데이터로더로 효율적인 학습하기

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_classification
from sklearn.model_selection import train_test_split

# 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 재현성을 위한 시드 설정
torch.manual_seed(42)
np.random.seed(42)

## 1. PyTorch 텐서 (Tensor)

PyTorch의 텐서는 NumPy 배열과 비슷하지만, GPU에서 연산이 가능하고 자동 미분을 지원합니다.

In [ ]:
# 텐서 생성
# NumPy 배열에서 텐서 생성
numpy_array = np.array([1, 2, 3, 4, 5])
tensor_from_numpy = torch.from_numpy(numpy_array)
print("NumPy에서 생성:", tensor_from_numpy)

# 직접 텐서 생성
tensor_direct = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0])
print("직접 생성:", tensor_direct)

# 특수 텐서들
zeros = torch.zeros(3, 4)
ones = torch.ones(2, 3)
random = torch.randn(3, 3)  # 표준정규분포

print("\n영텐서 (3x4):")
print(zeros)
print("\n무작위 텐서 (3x3):")
print(random)

### 1.1 텐서 연산

In [ ]:
# 기본 연산
a = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)
b = torch.tensor([[5, 6], [7, 8]], dtype=torch.float32)

print("텐서 a:")
print(a)
print("\n텐서 b:")
print(b)

# 요소별 연산
print("\n덧셈 (a + b):")
print(a + b)

print("\n곱셈 (a * b):")
print(a * b)

# 행렬 곱셈
print("\n행렬 곱셈 (a @ b):")
print(a @ b)  # 또는 torch.matmul(a, b)

# 통계 함수
print("\n평균:", a.mean())
print("표준편차:", a.std())
print("최댓값:", a.max())

### 1.2 자동 미분 (Autograd)

PyTorch의 가장 강력한 기능 중 하나는 자동 미분입니다!

In [ ]:
# requires_grad=True로 미분 가능한 텐서 생성
x = torch.tensor(2.0, requires_grad=True)
print(f"x = {x}")

# 연산 수행
y = x**2 + 3*x + 1
print(f"y = x² + 3x + 1 = {y}")

# 역전파
y.backward()

# 미분값 확인 (dy/dx = 2x + 3)
print(f"\ndy/dx at x=2: {x.grad}")
print(f"예상값: {2*2 + 3} (2x + 3)")

# 더 복잡한 예제
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x.sum()
z = y**2

print(f"\nx = {x}")
print(f"y = sum(x) = {y}")
print(f"z = y² = {z}")

z.backward()
print(f"\ndz/dx = {x.grad}")
print("(각 요소의 기여도가 2*sum(x) = 12)")

## 2. nn.Module로 신경망 구축하기

이제 Step 3에서 만든 MLP를 PyTorch로 다시 구현해봅시다!

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        
        # 레이어 정의
        self.fc1 = nn.Linear(input_size, hidden_size)  # 첫 번째 완전연결층
        self.relu = nn.ReLU()                          # ReLU 활성화 함수
        self.fc2 = nn.Linear(hidden_size, output_size) # 두 번째 완전연결층
        self.sigmoid = nn.Sigmoid()                    # 시그모이드 활성화 함수
    
    def forward(self, x):
        # 순전파 정의
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# 모델 생성
model = SimpleMLP(input_size=2, hidden_size=4, output_size=1)
print("모델 구조:")
print(model)

# 파라미터 확인
print("\n모델 파라미터:")
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}")

### 2.1 더 간결한 방법: nn.Sequential

In [ ]:
# Sequential을 사용한 동일한 모델
model_seq = nn.Sequential(
    nn.Linear(2, 4),
    nn.ReLU(),
    nn.Linear(4, 1),
    nn.Sigmoid()
)

print("Sequential 모델:")
print(model_seq)

# 더 복잡한 모델 예제
complex_model = nn.Sequential(
    nn.Linear(2, 16),
    nn.ReLU(),
    nn.Dropout(0.2),      # 드롭아웃 추가
    nn.Linear(16, 8),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(8, 1),
    nn.Sigmoid()
)

print("\n복잡한 모델 (드롭아웃 포함):")
print(complex_model)

## 3. XOR 문제 다시 해결하기

In [ ]:
# XOR 데이터 준비
X_xor = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y_xor = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# 모델, 손실 함수, 옵티마이저 정의
model_xor = nn.Sequential(
    nn.Linear(2, 4),
    nn.Tanh(),
    nn.Linear(4, 1),
    nn.Sigmoid()
)

criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.SGD(model_xor.parameters(), lr=0.5)

# 학습
losses = []
for epoch in range(1000):
    # 순전파
    outputs = model_xor(X_xor)
    loss = criterion(outputs, y_xor)
    
    # 역전파
    optimizer.zero_grad()  # 기울기 초기화
    loss.backward()        # 역전파
    optimizer.step()       # 가중치 업데이트
    
    losses.append(loss.item())
    
    if epoch % 200 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# 결과 시각화
plt.figure(figsize=(12, 4))

# 학습 곡선
plt.subplot(1, 3, 1)
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)

# 결정 경계
plt.subplot(1, 3, 2)
x_min, x_max = -0.5, 1.5
y_min, y_max = -0.5, 1.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))

grid_tensor = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)
with torch.no_grad():
    Z = model_xor(grid_tensor).numpy()
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, levels=20, alpha=0.8, cmap='RdBu')
plt.colorbar(label='Output')
plt.scatter([0, 1], [0, 1], c='red', s=200, edgecolors='black', linewidths=2)
plt.scatter([0, 1], [1, 0], c='blue', s=200, edgecolors='black', linewidths=2)
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('XOR Decision Boundary')
plt.grid(True, alpha=0.3)

# 예측 결과
plt.subplot(1, 3, 3)
with torch.no_grad():
    predictions = (model_xor(X_xor) > 0.5).float()
    probs = model_xor(X_xor)

table_data = [[x1, x2, y, pred.item(), f"{prob.item():.3f}"] 
              for (x1, x2), y, pred, prob in zip(X_xor.numpy(), y_xor.numpy(), predictions, probs)]
plt.table(cellText=table_data,
          colLabels=['x1', 'x2', 'Target', 'Prediction', 'Probability'],
          cellLoc='center',
          loc='center')
plt.axis('off')
plt.title('Predictions')

plt.tight_layout()
plt.show()

## 4. GPU 사용하기

PyTorch의 큰 장점 중 하나는 간단하게 GPU를 사용할 수 있다는 것입니다.

In [ ]:
# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 가능한 디바이스: {device}")

if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# 모델과 데이터를 GPU로 이동
model_gpu = nn.Sequential(
    nn.Linear(2, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid()
).to(device)  # GPU로 이동

# 데이터도 GPU로 이동
X_gpu = X_xor.to(device)
y_gpu = y_xor.to(device)

print(f"\n모델 위치: {next(model_gpu.parameters()).device}")
print(f"데이터 위치: {X_gpu.device}")

## 5. 데이터셋과 데이터로더

실제 딥러닝에서는 데이터를 효율적으로 관리하기 위해 Dataset과 DataLoader를 사용합니다.

In [ ]:
# 커스텀 데이터셋 클래스
class MoonsDataset(Dataset):
    def __init__(self, n_samples=1000, noise=0.2, train=True, test_size=0.2):
        # 데이터 생성
        X, y = make_moons(n_samples=n_samples, noise=noise, random_state=42)
        X = (X - X.mean(axis=0)) / X.std(axis=0)  # 정규화
        
        # 학습/테스트 분할
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42
        )
        
        if train:
            self.X = torch.tensor(X_train, dtype=torch.float32)
            self.y = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
        else:
            self.X = torch.tensor(X_test, dtype=torch.float32)
            self.y = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# 데이터셋 생성
train_dataset = MoonsDataset(n_samples=1000, train=True)
test_dataset = MoonsDataset(n_samples=1000, train=False)

print(f"학습 데이터 크기: {len(train_dataset)}")
print(f"테스트 데이터 크기: {len(test_dataset)}")

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 배치 확인
for batch_idx, (data, target) in enumerate(train_loader):
    print(f"\n배치 {batch_idx}: 입력 형태 = {data.shape}, 타겟 형태 = {target.shape}")
    if batch_idx == 2:  # 처음 3개 배치만 출력
        break

## 6. 완전한 학습 파이프라인 구축

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate=0.2):
        super(NeuralNetwork, self).__init__()
        
        layers = []
        prev_size = input_size
        
        # 은닉층 구성
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),  # 배치 정규화
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        
        # 출력층
        layers.append(nn.Linear(prev_size, output_size))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

# 학습 함수
def train_model(model, train_loader, test_loader, epochs=50, learning_rate=0.01):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    criterion = nn.BCEWithLogitsLoss()  # Sigmoid + BCE Loss
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)
    
    train_losses = []
    test_losses = []
    train_accs = []
    test_accs = []
    
    for epoch in range(epochs):
        # 학습 모드
        model.train()
        train_loss = 0
        train_correct = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            pred = (torch.sigmoid(output) > 0.5).float()
            train_correct += pred.eq(target).sum().item()
        
        # 평가 모드
        model.eval()
        test_loss = 0
        test_correct = 0
        
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += criterion(output, target).item()
                pred = (torch.sigmoid(output) > 0.5).float()
                test_correct += pred.eq(target).sum().item()
        
        # 평균 손실과 정확도 계산
        train_loss /= len(train_loader)
        test_loss /= len(test_loader)
        train_acc = train_correct / len(train_loader.dataset)
        test_acc = test_correct / len(test_loader.dataset)
        
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        train_accs.append(train_acc)
        test_accs.append(test_acc)
        
        # 학습률 조정
        scheduler.step(test_loss)
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f}, "
                  f"Test Loss = {test_loss:.4f}, Test Acc = {test_acc:.4f}")
    
    return train_losses, test_losses, train_accs, test_accs

# 모델 생성 및 학습
model = NeuralNetwork(input_size=2, hidden_sizes=[16, 8], output_size=1)
print("모델 구조:")
print(model)

train_losses, test_losses, train_accs, test_accs = train_model(
    model, train_loader, test_loader, epochs=50
)

In [ ]:
# 학습 결과 시각화
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 손실 그래프
axes[0, 0].plot(train_losses, label='Train Loss')
axes[0, 0].plot(test_losses, label='Test Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training and Test Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 정확도 그래프
axes[0, 1].plot(train_accs, label='Train Accuracy')
axes[0, 1].plot(test_accs, label='Test Accuracy')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].set_title('Training and Test Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 결정 경계 시각화
model.eval()
device = next(model.parameters()).device

# 전체 데이터 가져오기
X_all = torch.cat([train_dataset.X, test_dataset.X])
y_all = torch.cat([train_dataset.y, test_dataset.y])

# 결정 경계 계산
x_min, x_max = X_all[:, 0].min() - 0.5, X_all[:, 0].max() + 0.5
y_min, y_max = X_all[:, 1].min() - 0.5, X_all[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))

grid_tensor = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32).to(device)
with torch.no_grad():
    Z = torch.sigmoid(model(grid_tensor)).cpu().numpy()
Z = Z.reshape(xx.shape)

axes[1, 0].contourf(xx, yy, Z, levels=20, alpha=0.8, cmap='RdBu')
axes[1, 0].scatter(X_all[y_all.squeeze() == 0][:, 0], X_all[y_all.squeeze() == 0][:, 1], 
                   c='red', alpha=0.6, edgecolors='black', linewidths=1)
axes[1, 0].scatter(X_all[y_all.squeeze() == 1][:, 0], X_all[y_all.squeeze() == 1][:, 1], 
                   c='blue', alpha=0.6, edgecolors='black', linewidths=1)
axes[1, 0].set_xlabel('Feature 1')
axes[1, 0].set_ylabel('Feature 2')
axes[1, 0].set_title('Decision Boundary')
axes[1, 0].grid(True, alpha=0.3)

# 예측 확률 분포
with torch.no_grad():
    probs = torch.sigmoid(model(X_all.to(device))).cpu().numpy()

axes[1, 1].hist(probs[y_all.squeeze() == 0], bins=20, alpha=0.5, label='Class 0', color='red')
axes[1, 1].hist(probs[y_all.squeeze() == 1], bins=20, alpha=0.5, label='Class 1', color='blue')
axes[1, 1].set_xlabel('Predicted Probability')
axes[1, 1].set_ylabel('Count')
axes[1, 1].set_title('Prediction Probability Distribution')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 다양한 최적화 기법

In [ ]:
# 다양한 옵티마이저 비교
optimizers = {
    'SGD': optim.SGD,
    'Adam': optim.Adam,
    'RMSprop': optim.RMSprop,
    'AdamW': optim.AdamW
}

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for idx, (name, optimizer_class) in enumerate(optimizers.items()):
    # 모델 초기화
    model = nn.Sequential(
        nn.Linear(2, 16),
        nn.ReLU(),
        nn.Linear(16, 8),
        nn.ReLU(),
        nn.Linear(8, 1),
        nn.Sigmoid()
    )
    
    criterion = nn.BCELoss()
    optimizer = optimizer_class(model.parameters(), lr=0.01)
    
    # 학습
    losses = []
    for epoch in range(100):
        epoch_loss = 0
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        losses.append(epoch_loss / len(train_loader))
    
    axes[idx].plot(losses)
    axes[idx].set_xlabel('Epoch')
    axes[idx].set_ylabel('Loss')
    axes[idx].set_title(f'{name} Optimizer')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. 모델 저장과 불러오기

In [ ]:
# 모델 저장
# 방법 1: 전체 모델 저장
torch.save(model, 'complete_model.pth')

# 방법 2: 가중치만 저장 (권장)
torch.save(model.state_dict(), 'model_weights.pth')

# 체크포인트 저장 (학습 재개용)
checkpoint = {
    'epoch': 50,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': train_losses[-1],
}
torch.save(checkpoint, 'checkpoint.pth')

print("모델 저장 완료!")

# 모델 불러오기
# 방법 1: 전체 모델 불러오기
loaded_model = torch.load('complete_model.pth')

# 방법 2: 가중치만 불러오기 (권장)
new_model = NeuralNetwork(input_size=2, hidden_sizes=[16, 8], output_size=1)
new_model.load_state_dict(torch.load('model_weights.pth'))
new_model.eval()  # 평가 모드로 전환

print("\n모델 불러오기 완료!")

# 불러온 모델로 예측
with torch.no_grad():
    sample_data = torch.tensor([[0.5, 0.5]], dtype=torch.float32)
    prediction = torch.sigmoid(new_model(sample_data))
    print(f"\n샘플 예측: 입력 = {sample_data.numpy()}, 출력 = {prediction.item():.4f}")

## 9. 실전 팁과 트릭

In [ ]:
# 1. 가중치 초기화
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

model = nn.Sequential(
    nn.Linear(2, 16),
    nn.ReLU(),
    nn.Linear(16, 1)
)
model.apply(init_weights)
print("가중치 초기화 완료")

# 2. 그래디언트 클리핑
def train_with_gradient_clipping(model, data_loader, epochs=10):
    optimizer = optim.Adam(model.parameters())
    criterion = nn.MSELoss()
    
    for epoch in range(epochs):
        for data, target in data_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            
            # 그래디언트 클리핑
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            break  # 데모용으로 첫 배치만 실행

# 3. 조기 종료 (Early Stopping)
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# 사용 예시
early_stopping = EarlyStopping(patience=5)
print("Early Stopping 준비 완료")

# 4. 학습률 스케줄링
model = nn.Linear(10, 1)
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 다양한 스케줄러
scheduler_step = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
scheduler_exp = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
scheduler_cosine = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

print("학습률 스케줄러 준비 완료")

## 10. 연습 문제

In [ ]:
# 문제 1: 다중 클래스 분류 모델 구현
# make_classification을 사용하여 3개 클래스 분류 문제를 해결하세요
def create_multiclass_model(input_size, num_classes):
    # 힌트: 마지막 층은 nn.Linear(?, num_classes)
    # 손실 함수는 nn.CrossEntropyLoss() 사용
    pass

# 문제 2: 커스텀 활성화 함수 구현
class Swish(nn.Module):
    def forward(self, x):
        # Swish(x) = x * sigmoid(x)
        pass

# 문제 3: 앙상블 모델 구현
# 여러 모델의 예측을 평균내는 앙상블 클래스를 만드세요
class EnsembleModel(nn.Module):
    def __init__(self, models):
        super().__init__()
        self.models = nn.ModuleList(models)
    
    def forward(self, x):
        # 모든 모델의 출력을 평균
        pass

## 정리

이번 튜토리얼에서 배운 내용:
1. PyTorch 텐서와 자동 미분
2. nn.Module을 사용한 신경망 구축
3. 손실 함수와 옵티마이저
4. GPU 활용
5. Dataset과 DataLoader로 효율적인 데이터 처리
6. 모델 저장과 불러오기
7. 다양한 최적화 기법과 팁

### PyTorch의 장점:
- **동적 계산 그래프**: 디버깅이 쉽고 직관적
- **Pythonic**: Python다운 코드 작성 가능
- **강력한 생태계**: torchvision, torchaudio 등 다양한 도구
- **연구와 프로덕션**: 연구부터 배포까지 모두 가능

다음 단계에서는 CNN을 사용하여 이미지 분류를 해보겠습니다!